Based on 4chan NovelAILeaks (naifu) [link](https://boards.4channel.org/g/thread/89095460#p89097704)


**adapted by physics1024 for kaggle, thanks for Anonymous, 炼铜术士, 神楽坂早苗, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb, JingShing**

In [ ]:
# Check GPU working status

!nvidia-smi

In [ ]:
# 1. Download Novel AI API backend, model
# If the download speed is too slow try restart

!mkdir ~/content
%cd ~/content/
!apt install -y aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm -rf naifu.tar
!echo "Done."

In [ ]:
# 2. Install dependencies
# Wait patiently for the installation to complete

%cd ~/content/naifu
!pip install virtualenv && bash ./setup.sh
!aria2c -o bore.tar.gz https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz
!tar zxf bore.tar.gz -C /usr/bin && rm -rf bore.tar.gz
!aria2c -o cloudflared -d /usr/bin https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

**notice that if you want to run 7G version, *do not* run the cell beneath but the cell after it**

In [ ]:
# 3. Start the model
# Just visit the output mapping address (end with `trycloudflare.com` / `bore.pub`).
# - Please wait until the model is loaded (`Application startup complete` appears) before accessing
# - The service provided by cloudflare occasionally has a request timeout, which can be replaced by bore tunnel

%cd ~/content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

**the cell below is the 7G version**

In [ ]:
# (Optional) Run with `animefull-latest` model of 7G
# uses the 4G size animefull-final-pruned model by default. If you want to use the 7G animefull-latest model, run this

%cd ~/content/
!tar xf animefull-latest.tar -C ~/content/naifu/models && rm -rf animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' ~/content/naifu/hydra_node/models.py

%cd ~/content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969